# Object-based landuse classification

## Introduction

[link](https://towardsdatascience.com/object-based-land-cover-classification-with-python-cbe54e9c9e24)

Aerial images cover the entire globe at various spatial and temporal resolutions. Timely extraction of information from aerial images requires automated analysis to train computers to recognize what the human eye immediately identifies. Object-based image analysis (OBIA) improves processing efficiency by implementing image segmentation algorithms to combine groups of pixels into objects (segments) reducing the amount of information in and image. This article describes how to use open source Python packages to perform image segmentation and land cover classification of an aerial image. Specifically, I will demonstrate the process of geographic object-based image analysis (GeOBIA)to perform supervised land cover classification in 5 steps:

- Image segmentation
- Quantify segment spectral properties
- Truth data
- Land cover classification
- Accuracy assessment

## Image segmentation

The image above is a portion of an aerial photo collected by the US Department of Agriculture (USDA) under the National Agricultural Imagery Progam (NAIP). The horizontal image resolution is 1 meter. Our first task is to group similar pixels into segments. Segmentation effectively reduces the number of elements in an image that need to be classified. This may reduce an image with 1 million pixels down to 50,000 segments, which is much more manageable.

A number of segmentation algorithms are available. We won’t go into details in detail in this article. I will show you results from two different algorithms, and how to implement them in Python with `skimage`.

[Here](https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_segmentations.html#sphx-glr-auto-examples-segmentation-plot-segmentations-py) you can read more about image segmentation.

The code below demonstrates segmentation with the SLIC (Simple linear iterative clustering) and quickshift algorithms (lines 23 and 24, respectively). First, each of the 4 bands (red, blue, green, near-infrared) from the NAIP image is read as a `numpy` array with `gdal`. Band data are re-scaled to intensity values (ranging from 0–1). Then segments are created. Segments are saved to a new raster with `gdal`.

In [ ]:
import numpy as np
from osgeo import gdal
from osgeo import ogr
from skimage import exposure
from skimage.segmentation import quickshift
import geopandas as gpd
import numpy as np
 
naip_fn = 'path/to/image.tif'
 
driverTiff = gdal.GetDriverByName('GTiff')
naip_ds = gdal.Open(naip_fn)
nbands = naip_ds.RasterCount
band_data = []

for i in range(1, nbands+1):
    band = naip_ds.GetRasterBand(i).ReadAsArray()
    band_data.append(band)
band_data = np.dstack(band_data)
img = exposure.rescale_intensity(band_data)

# do segmentation, different options with quickshift and slic (only use one of the next two lines)
segments = quickshift(img, ratio=0.99, max_dist=5, convert2lab=False)
segments = slic(img, n_segments=500000, compactness=0.1)
print('segments complete', time.time() - seg_start)
 
# save segments to raster
segments_fn = 'path/to/segments.tif'
segments_ds = driverTiff.Create(segments_fn, naip_ds.RasterXSize, naip_ds.RasterYSize,
                                1, gdal.GDT_Float32)
segments_ds.SetGeoTransform(naip_ds.GetGeoTransform())
segments_ds.SetProjection(naip_ds.GetProjectionRef())
segments_ds.GetRasterBand(1).WriteArray(segments)
segments_ds = None

Images below demonstrate how the segmentation algorithm and parameters affect the size and shape of the image segments. Segments in the first image (quickshift) don’t capture the features in the image very well. You see segments overlapping roads, fields, and forested areas. In the second image, the segments from the SLIC algorithm do a much better following the boundaries of image features. The remainder of this article uses the SLIC result. Features in your image may be best represented by a different algorithm, or different algorithm parameters. Be sure to assess your segments before continuing with classification.

## Spectral Properties of Image Segments

Once the image is segmented the spectral properties of each segment must be quantitatively described. Given a number of pixels, the function below calculates descriptive statistics (e.g. mean, max, min, variance) for each band. These are the values that will be used by the random forests algorithm to classify the segments into land cover types.



In [ ]:
def segment_features(segment_pixels):
    features = []
    npixels, nbands = segment_pixels.shape
    for b in range(nbands):
        stats = scipy.stats.describe(segment_pixels[:, b])
        band_stats = list(stats.minmax) + list(stats)[2:]
        if npixels == 1:
            # in this case the variance = nan, change it 0.0
            band_stats[3] = 0.0
        features += band_stats
    return features

Now, we loop through each segment, send the pixels from each segment to the `segment_features` function and save the results in a list. This section of code will likely be the bottleneck in your processing time. It can be improved with parallelization, but that isn’t discussed here.

In [ ]:
segment_ids = np.unique(segments)
objects = []
object_ids = []
for id in segment_ids:
    segment_pixels = img[segments == id]
    object_features = segment_features(segment_pixels)
    objects.append(object_features)
    object_ids.append(id)

## Truth (Training and Test) Data

This is a supervised classification workflow, so you’ll need to have some truth data describing the land cover types represented in your classification. I quickly generated the points below in QGIS to represent seven different land cover classes. These data are just an example. Ideally, you would have data collected in a more organized and statistically rigorous manner.

The land cover truth data need to be split into training and test data sets. The training data set will train the random forests classification algorithm. We will compare the classification results to the test data set to assess classification accuracy.

My land cover data are in shapefile format. The code below uses `geopandas` to read the truth data as a geodataframe. Randomly, 70% of the truth observations are assigned to a training data set and the remaining 30% to a testing data set. The training and test data sets are each saved to a new shapefile. During this process I also used a lookup table that I created to give names to each land cover class (lines 8–11). This is not necessary, but makes it easier to see what each class represents.

In [ ]:
# read shapefile to geopandas geodataframe
gdf = gpd.read_file('path/to/truth_data.shp')
# get names of land cover classes/labels
class_names = gdf['label'].unique()
# create a unique id (integer) for each land cover class/label
class_ids = np.arange(class_names.size) + 1
# create a pandas data frame of the labels and ids and save to csv
df = pd.DataFrame({'label': class_names, 'id': class_ids})
df.to_csv('C:/temp/naip/class_lookup.csv')
# add a new column to geodatafame with the id for each class/label
gdf['id'] = gdf['label'].map(dict(zip(class_names, class_ids)))
 
# split the truth data into training and test data sets and save each to a new shapefile
gdf_train = gdf.sample(frac=0.7)  # 70% of observations assigned to training data (30% to test data)
gdf_test = gdf.drop(gdf_train.index)
# save training and test data to shapefiles
gdf_train.to_file('path/to/save/train_data.shp')
gdf_test.to_file('path/to/save/test_data.shp')

Now, convert the training data to raster format so each observation point can be associated with an image segment.

In [ ]:
train_fn = 'path/to/train_data.shp'
train_ds = ogr.Open(train_fn)
lyr = train_ds.GetLayer()
# create a new raster layer in memory
driver = gdal.GetDriverByName('MEM')
target_ds = driver.Create('', naip_ds.RasterXSize, naip_ds.RasterYSize, 1, gdal.GDT_UInt16)
target_ds.SetGeoTransform(naip_ds.GetGeoTransform())
target_ds.SetProjection(naip_ds.GetProjection())
# rasterize the training points
options = ['ATTRIBUTE=id']
gdal.RasterizeLayer(target_ds, [1], lyr, options=options)
# retrieve the rasterized data and print basic stats
data = target_ds.GetRasterBand(1).ReadAsArray()

Associate each training observation with the corresponding image segment. Lines 13–19 ensure that each training observation is associated with only one segment. Because segments include multiple pixels, it is possible that segments represent multiple land cover types. This is why it is important to properly tune your segmentation algorithm.



In [ ]:
# rasterized observation (truth, training and test) data
ground_truth = target_ds.GetRasterBand(1).ReadAsArray()

# get unique values (0 is the background, or no data, value so it is not included) for each land cover type
classes = np.unique(ground_truth)[1:]

# for each class (land cover type) record the associated segment IDs
segments_per_class = {}
for klass in classes:
    segments_of_class = segments[ground_truth == klass]
    segments_per_class[klass] = set(segments_of_class)
 
# make sure no segment ID represents more than one class
intersection = set()
accum = set()
for class_segments in segments_per_class.values():
    intersection |= accum.intersection(class_segments)
    accum |= class_segments
assert len(intersection) == 0, "Segment(s) represent multiple classes"

## Land Cover Classification

This is the meat of the analysis. The classification algorithm. First, identify and label the training objects (lines 1–20). This process involves associating a label (land cover type) with the statistics describing each spectral band within the image segment.

Now, everything is now set up to train a classifier and use it to predict across all segments in the image. Here I’m using random forests, a popular classification algorithm. The code to train (fit) the algorithm and make predictions is quite simple (lines 22–24). Simply pass the training objects (containing the spectral properties) and the associated land cover label to the classifier. Once the classifier is trained (fitted) predictions can be made for non-training segments based on their spectral properties. After the predictions are made, save them to raster for display in a GIS (lines 26–43).

In [ ]:
train_img = np.copy(segments)
threshold = train_img.max() + 1  # make the threshold value greater than any land cover class value

# all pixels in training segments assigned value greater than threshold
for klass in classes:
    class_label = threshold + klass
    for segment_id in segments_per_class[klass]:
        train_img[train_img == segment_id] = class_label
 
# training segments receive land cover class value, all other segments 0
train_img[train_img <= threshold] = 0
train_img[train_img > threshold] -= threshold

# create objects and labels for training data
training_objects = []
training_labels = []
for klass in classes:
    class_train_object = [v for i, v in enumerate(objects) if segment_ids[i] in segments_per_class[klass]]
    training_labels += [klass] * len(class_train_object)
    training_objects += class_train_object
 
classifier = RandomForestClassifier(n_jobs=-1)  # setup random forest classifier
classifier.fit(training_objects, training_labels)  # fit rf classifier
predicted = classifier.predict(objects)  # predict with rf classifier

# create numpy array from rf classifiation and save to raster
clf = np.copy(segments)
for segment_id, klass in zip(segment_ids, predicted):
    clf[clf == segment_id] = klass
 
mask = np.sum(img, axis=2)  # this section masks no data values
mask[mask > 0.0] = 1.0
mask[mask == 0.0] = -1.0
clf = np.multiply(clf, mask)
clf[clf < 0] = -9999.0
 
clfds = driverTiff.Create('path/to/classified_result.tif', naip_ds.RasterXSize, naip_ds.RasterYSize,
                          1, gdal.GDT_Float32)  # this section saves to raster
clfds.SetGeoTransform(naip_ds.GetGeoTransform())
clfds.SetProjection(naip_ds.GetProjection())
clfds.GetRasterBand(1).SetNoDataValue(-9999.0)
clfds.GetRasterBand(1).WriteArray(clf)
clfds = None
 
print('Done!')

## Accuracy Assessment with a Confusion Matrix

Accuracy assessment is a crucial aspect of any classification. If your classification doesn’t represent what it’s supposed to, it’s not worth much. Because the emphasis of this article is to describe the GeOBIA workflow, I’m not presenting my own accuracy results. Instead, I’ll show how to generate a basic confusion matrix for accuracy assessment.

Load the test data set created earlier and convert it to raster format so it is compatible with the generated predictions. Then simply query the predicted values from the locations where test data exist. Finally, generate the confusion matrix from the corresponding values.

Note: We classified segments, but this accuracy assessment compares pixels. We’re comparing all the pixels in each test segment to all the pixels in the corresponding predicted segment. This could lead to some bias if certain land cover classes are more frequently found in smaller (or larger) segments than others. Again, if you have done due diligence with image segmentation, this shouldn’t be a problem.

In [ ]:
import numpy as np
import gdal
import ogr
from sklearn import metrics
 
# read original image to get info for raster dimensions
naip_fn = 'path/to/image.tif'
driverTiff = gdal.GetDriverByName('GTiff')
naip_ds = gdal.Open(naip_fn)
 
# rasterize test data for pixel-to-pixel comparison
test_fn = 'path/to/test.shp'
test_ds = ogr.Open(test_fn)
lyr = test_ds.GetLayer()
driver = gdal.GetDriverByName('MEM')
target_ds = driver.Create('', naip_ds.RasterXSize, naip_ds.RasterYSize, 1, gdal.GDT_UInt16)
target_ds.SetGeoTransform(naip_ds.GetGeoTransform())
target_ds.SetProjection(naip_ds.GetProjection())
options = ['ATTRIBUTE=id']
gdal.RasterizeLayer(target_ds, [1], lyr, options=options)
 
truth = target_ds.GetRasterBand(1).ReadAsArray()  # truth/test data array
 
pred_ds = gdal.Open('path/to/classified_result.tif')  
pred = pred_ds.GetRasterBand(1).ReadAsArray()  # predicted data array
idx = np.nonzero(truth) # get indices where truth/test has data values
cm = metrics.confusion_matrix(truth[idx], pred[idx])  # create a confusion matrix at the truth/test locations
 
# pixel accuracy
print(cm)
print(cm.diagonal())
print(cm.sum(axis=0))
accuracy = cm.diagonal() / cm.sum(axis=0)  # overall accuracy
print(accuracy)